Rahmanda Afebrio Yuris Soesatyo - Chapter 19:  Training and Deploying TensorFlow Models at Scale

Chapter 19 - Training and Deploying TensorFlow Models at Scale

1. SavedModel dan TensorFlow Serving
Setelah pelatihan, model tf.keras biasanya diekspor ke format SavedModel, yang menyimpan:
graph komputasi,
bobot model,
aset tambahan (mis. vocabulary atau contoh data).
SavedModel berbentuk folder dengan struktur utama:
saved_model.pb
variables/ (file bobot)
assets/ (opsional).
Model dapat dimuat kembali menggunakan:
tf.saved_model.load() atau
keras.models.load_model().
TensorFlow Serving adalah server C++ berperforma tinggi untuk deployment model, dengan fitur:
serving multi-versi model,
auto-reload dan rollback versi,
batching request otomatis.
Model dapat diakses melalui REST (HTTP JSON) atau gRPC, dan mudah dijalankan menggunakan Docker.

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

# 1) Train simple Keras model (contoh MNIST)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1)

# 2) Export to SavedModel (versi 0001)
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path)
# atau: model.save(model_path)  # juga SavedModel jika tanpa ekstensi .h5


In [ ]:
# 3) Query TF Serving via REST
import json
import requests
import numpy as np

X_new = X_test[:3]  # misal 3 gambar uji
input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

SERVER_URL = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status()
response = response.json()
y_proba = np.array(response["predictions"])
print(y_proba.round(2))

In [ ]:
#  4) Query TF Serving via gRPC
import grpc
import tensorflow as tf
from tensorflow_serving.apis.predict_pb2 import PredictRequest
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel("localhost:8500")
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

response = predict_service.Predict(request, timeout=10.0)
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
print(y_proba.round(2))


2. Google Cloud AI Platform: Prediction Service & Hyperparameter Tuning

Google Cloud AI Platform (Vertex AI) menyediakan layanan hosting prediction berbasis TensorFlow Serving dengan:
autoscaling,
logging,
integrasi langsung dengan Google Cloud Storage (GCS).
Alur deployment umum:
Upload SavedModel ke GCS.
Buat Model di AI Platform.
Buat Version yang menunjuk ke path SavedModel
(mis. gs://bucket/model_name/0001/).
Akses prediction dilakukan melalui REST API menggunakan Google API Client Library.
Autentikasi menggunakan service account dengan key JSON yang diset melalui GOOGLE_APPLICATION_CREDENTIALS.
AI Platform mendukung hyperparameter tuning berbasis Bayesian optimization (Google Vizier), dikonfigurasi lewat file YAML dan menggunakan metrik dari TensorBoard (misalnya accuracy atau loss).

In [ ]:
import os
import numpy as np
import googleapiclient.discovery

# 1) Autentikasi service account
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

# 2) Bangun resource untuk AI Platform
project_id = "your-gcp-project-id"
model_id = "my_mnist_model"
model_path = f"projects/{project_id}/models/{model_id}"

ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

output_name = "dense_1"  # sesuaikan dengan nama output layer SavedModel

def predict(X):
    body = {
        "signature_name": "serving_default",
        "instances": X.tolist(),
    }
    request = ml_resource.predict(name=model_path, body=body)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred[output_name] for pred in response["predictions"]])

Y_probas = predict(X_new)
print(np.round(Y_probas, 2))


In [ ]:
# Di training script: gunakan argumen CLI & TensorBoard callback
import argparse
import tensorflow as tf
from tensorflow import keras

parser = argparse.ArgumentParser()
parser.add_argument("--n_layers", type=int, default=20)
parser.add_argument("--momentum", type=float, default=0.9)
parser.add_argument("--job-dir", type=str, default="/tmp/jobdir")
args, _ = parser.parse_known_args()

model = keras.models.Sequential(
    [keras.layers.Flatten(input_shape=[28, 28])] +
    [keras.layers.Dense(100, activation="relu") for _ in range(args.n_layers)] +
    [keras.layers.Dense(10, activation="softmax")]
)
optimizer = keras.optimizers.SGD(momentum=args.momentum)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

tb_cb = keras.callbacks.TensorBoard(
    log_dir=args.job_dir,
    update_freq="batch"
)

model.fit(X_train, y_train, epochs=10, validation_split=0.1,
          callbacks=[tb_cb])

3. TFLite dan TF.js: Deployment ke Mobile, Embedded, dan Browser

Bagian ini membahas TensorFlow Lite (TFLite) dan TensorFlow.js sebagai solusi deployment model ke platform dengan keterbatasan sumber daya seperti perangkat mobile, embedded system, dan browser. TFLite dirancang agar model bisa berjalan dengan ukuran file yang jauh lebih kecil, latensi rendah, serta konsumsi energi yang hemat, sehingga cocok untuk hardware dengan keterbatasan RAM dan CPU. Model yang awalnya berupa SavedModel atau tf.keras dikonversi menjadi format .tflite berbasis FlatBuffers menggunakan TFLite Converter, yang sekaligus menerapkan berbagai optimisasi graph seperti menghapus operasi yang hanya dibutuhkan saat training, menggabungkan Batch Normalization, dan menyederhanakan struktur komputasi.

Untuk meningkatkan efisiensi lebih jauh, TFLite mendukung post-training quantization, misalnya mengubah representasi bobot dari float32 ke int8. Teknik ini secara signifikan mengecilkan ukuran model dan mempercepat inference. Pada skema kuantisasi simetris, nilai float dalam rentang tertentu dipetakan ke rentang integer terbatas, sehingga memungkinkan full integer inference dengan latensi yang sangat rendah. Jika penurunan akurasi menjadi perhatian, quantization-aware training (QAT) dapat digunakan dengan menyimulasikan efek kuantisasi selama training, sehingga model lebih siap saat dikonversi ke format TFLite.

Sementara itu, TensorFlow.js memungkinkan model dijalankan langsung di browser menggunakan format model.json dan shard bobot biner. Pendekatan ini sangat cocok untuk aplikasi web yang membutuhkan latensi rendah, kemampuan berjalan secara offline, serta perlindungan privasi karena inference dilakukan sepenuhnya di sisi klien tanpa harus mengirim data ke server. Dengan dukungan TFLite dan TF.js, TensorFlow menyediakan jalur deployment end-to-end dari training hingga inference di berbagai platform nyata.

In [ ]:
import tensorflow as tf

# Asumsikan model Keras sudah dilatih
saved_model_path = "my_mnist_model/0001"

# 1) Konversi SavedModel ke TFLite basic
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()
with open("model_basic.tflite", "wb") as f:
    f.write(tflite_model)

# 2) Post-training quantization (OPTIMIZE_FOR_SIZE)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
with open("model_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)

# 3) Jalankan TFLite model di Python dengan TFLite Interpreter
import numpy as np

interpreter = tf.lite.Interpreter(model_path="model_quant.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

X_sample = X_test[:1].astype(np.float32)
interpreter.set_tensor(input_details[0]["index"], X_sample)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]["index"])
print(output_data.round(2))

4. GPU/TPU, Manajemen Memori, dan Multi-Device Placement

Melatih deep model di CPU umumnya terasa lambat karena operasi utama seperti convolution dan perkalian matriks sangat intensif secara komputasi. GPU dan TPU dirancang khusus untuk jenis operasi ini, sehingga mampu mempercepat proses training dan inference secara signifikan. Karena itu, penggunaan akselerator menjadi standar dalam praktik deep learning modern.

Untuk menjalankan TensorFlow dengan GPU lokal, diperlukan beberapa komponen pendukung, yaitu driver NVIDIA, CUDA, dan cuDNN yang versinya saling kompatibel. Setelah terpasang, ketersediaan GPU bisa dicek langsung dari TensorFlow, misalnya dengan fungsi tf.test.is_gpu_available() atau dengan melihat daftar device fisik melalui tf.config.experimental.list_physical_devices('GPU').

Jika tidak ingin repot dengan instalasi lokal, alternatif yang jauh lebih praktis adalah menggunakan Google Colab yang menyediakan GPU dan TPU gratis, atau menyewa cloud VM berbasis GPU seperti Google Cloud Deep Learning VM. Opsi-opsi ini sering dipakai untuk eksperimen, tugas kuliah, maupun training model berskala besar.

Secara default, TensorFlow akan langsung mengalokasikan seluruh memori dari GPU pertama yang terdeteksi, yang kadang menyulitkan jika GPU ingin dipakai bersama proses lain. Untuk mengatasinya, tersedia pengaturan seperti set_virtual_device_configuration dan set_memory_growth yang memungkinkan pembatasan penggunaan memori GPU, alokasi memori secara bertahap sesuai kebutuhan, serta pembagian satu GPU untuk beberapa proses.

Dalam hal penempatan komputasi, TensorFlow biasanya secara otomatis menentukan apakah sebuah operasi atau variabel dijalankan di CPU atau GPU. Namun, jika dibutuhkan kontrol lebih spesifik, penempatan ini bisa dioverride secara manual menggunakan konteks tf.device, misalnya memaksa eksekusi ke /gpu:1 atau /cpu:0 sesuai kebutuhan eksperimen atau debugging.

In [ ]:
import tensorflow as tf

# 1) Cek GPU
print(tf.config.experimental.list_physical_devices("GPU"))

# 2) Set memory growth supaya TF tidak grab semua RAM GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 3) Batasi RAM GPU (misal 2 GiB per virtual device)
for gpu in gpus:
    tf.config.experimental.set_virtual_device_configuration(
        gpu,
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
    )

# 4) Atau split 1 GPU menjadi 2 virtual GPU (2 GiB + 2 GiB)
physical_gpus = tf.config.experimental.list_physical_devices("GPU")
if physical_gpus:
    tf.config.experimental.set_virtual_device_configuration(
        physical_gpus[0],
        [
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
            tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
        ],
    )
    logical_gpus = tf.config.experimental.list_logical_devices("GPU")
    print("Logical GPUs:", logical_gpus)


In [ ]:
# 5) Contoh penempatan variabel di CPU vs GPU
a = tf.Variable(42.0)  # float → GPU default bila ada
b = tf.Variable(42)    # int → CPU
print("a on:", a.device)
print("b on:", b.device)

with tf.device("/cpu:0"):
    c = tf.Variable(3.14)
print("c on:", c.device)


5. Distribution Strategies: Multi-GPU & Multi-Server Training

Pada skala besar, proses training biasanya dipercepat dengan data parallelism, yaitu dengan mereplikasi model yang sama ke beberapa device. Setiap device memproses mini-batch data yang berbeda, lalu gradien yang dihasilkan digabungkan—umumnya menggunakan mekanisme seperti AllReduce—sebelum parameter model diperbarui. Dengan cara ini, beban komputasi terbagi rata tanpa mengubah arsitektur model secara fundamental.

Untuk kasus multi-GPU dalam satu mesin, TensorFlow menyediakan MirroredStrategy. Strategi ini membuat replika model di setiap GPU dan secara otomatis melakukan agregasi gradien menggunakan AllReduce berbasis NCCL. Pendekatan ini relatif sederhana untuk digunakan dan sangat efisien pada single-node dengan banyak GPU, sehingga sering menjadi pilihan default untuk training paralel lokal.

Alternatif lain adalah CentralStorageStrategy, di mana seluruh parameter model disimpan di satu device, biasanya CPU, sementara GPU hanya bertugas menghitung gradien. Strategi ini cocok untuk model yang relatif kecil atau pada kondisi di mana memori GPU terbatas, meskipun performanya umumnya tidak secepat MirroredStrategy karena adanya bottleneck pada device pusat.

Untuk training terdistribusi lintas banyak mesin, TensorFlow mendukung MultiWorkerMirroredStrategy dan ParameterServerStrategy. Konfigurasi cluster didefinisikan melalui variabel environment TF_CONFIG, yang menetapkan peran seperti chief, worker, parameter server, dan evaluator. MultiWorkerMirroredStrategy pada dasarnya memperluas MirroredStrategy ke level multi-server dengan sinkronisasi gradien lintas mesin menggunakan AllReduce, sehingga cocok untuk training sinkron berskala besar.

Sebaliknya, ParameterServerStrategy memisahkan peran komputasi dan penyimpanan parameter: worker bertugas menghitung gradien, sementara parameter server menyimpan dan memperbarui bobot model. Pendekatan ini lebih fleksibel dan sering dipakai pada cluster besar atau skenario asinkron, meskipun arsitekturnya lebih kompleks. Dalam praktiknya, job distributed seperti ini sering dijalankan di cloud, misalnya menggunakan layanan seperti Vertex AI.

Terlepas dari strategi distribusi yang digunakan saat training, model hasil akhirnya tetap disimpan dalam format SavedModel standar. Model ini dapat diload dan dijalankan tanpa konteks distribusi sama sekali, atau kembali dijalankan dengan distribution strategy jika ingin melakukan inference paralel di lingkungan multi-GPU.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 1) Buat strategy di semua GPU lokal
strategy = tf.distribute.MirroredStrategy()  # atau MirroredStrategy(["/gpu:0","/gpu:1"])
print("Num replicas in sync:", strategy.num_replicas_in_sync)

# 2) Bangun & compile model di dalam scope strategy
with strategy.scope():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.Adam(),
                  metrics=["accuracy"])

# Batch size harus kelipatan jumlah replika
global_batch_size = 256
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=global_batch_size,
                    validation_split=0.1)

# 3) Simpan model seperti biasa
model.save("mnist_multi_gpu.h5")

# 4) Load untuk inference saja (single device)
single_model = keras.models.load_model("mnist_multi_gpu.h5")
y_proba_single = single_model.predict(X_test[:512])

# 5) Atau load lagi dalam scope untuk inference multi-GPU
with strategy.scope():
    dist_model = keras.models.load_model("mnist_multi_gpu.h5")
y_proba_dist = dist_model.predict(X_test[:global_batch_size])

In [ ]:

# Example skeleton: CentralStorageStrategy
strategy = tf.distribute.experimental.CentralStorageStrategy()

with strategy.scope():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

model.fit(X_train, y_train,
          epochs=5,
          batch_size=global_batch_size,
          validation_split=0.1)
